In [21]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('Car Launch').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 14:03:21 WARN Utils: Your hostname, Pulastyas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.40 instead (on interface en0)
26/01/19 14:03:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 14:03:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.options(header=True, inferSchema=True).csv('data/input/car_launch.csv')

In [6]:
df.show()

+----+------------+------------+
|year|company_name|product_name|
+----+------------+------------+
|2019|      Toyota|      Avalon|
|2019|      Toyota|       Camry|
|2020|      Toyota|     Corolla|
|2019|       Honda|      Accord|
|2019|       Honda|    Passport|
|2019|       Honda|        CR-V|
|2020|       Honda|       Pilot|
|2019|       Honda|       Civic|
|2020|   Chevrolet| Trailblazer|
|2020|   Chevrolet|        Trax|
|2019|   Chevrolet|    Traverse|
|2020|   Chevrolet|      Blazer|
|2019|        Ford|        Figo|
|2020|        Ford|      Aspire|
|2019|        Ford|   Endeavour|
|2020|        Jeep|    Wrangler|
|2020|        Jeep|    Cherokee|
|2020|        Jeep|     Compass|
|2019|        Jeep|    Renegade|
|2019|        Jeep|   Gladiator|
+----+------------+------------+



In [15]:
result = df \
    .groupBy('company_name') \
    .pivot('year') \
    .agg(F.count('product_name')) \
    .withColumn('net_products', F.col('2020') - F.col('2019')) \
    .select('company_name', 'net_products')
    
result.show()

+------------+------------+
|company_name|net_products|
+------------+------------+
|        Jeep|           1|
|   Chevrolet|           2|
|       Honda|          -3|
|      Toyota|          -1|
|        Ford|          -1|
+------------+------------+



In [17]:
login = spark.read.options(header=True, inferSchema=True).csv('data/input/user_login.csv')

login.show()

+-----------+----------+-------+
|record_date|account_id|user_id|
+-----------+----------+-------+
| 2021-01-01|        A1|     U1|
| 2021-01-01|        A1|     U2|
| 2021-01-06|        A1|     U3|
| 2021-01-02|        A1|     U1|
| 2020-12-24|        A1|     U2|
| 2020-01-03|        A1|     U1|
| 2020-12-09|        A1|     U1|
| 2021-01-10|        A2|     U4|
| 2021-01-11|        A2|     U4|
| 2021-01-12|        A2|     U4|
| 2021-01-15|        A2|     U5|
| 2020-12-17|        A2|     U4|
| 2020-12-25|        A3|     U6|
| 2020-12-25|        A3|     U6|
| 2020-12-25|        A3|     U6|
| 2020-12-06|        A3|     U7|
| 2020-12-06|        A3|     U6|
| 2021-01-14|        A3|     U6|
| 2021-02-07|        A1|     U1|
| 2021-02-10|        A1|     U2|
+-----------+----------+-------+
only showing top 20 rows


In [25]:
result1 = login.dropDuplicates()\
    .withColumn('record_date', F.to_date('record_date', 'YYYY-MM-DD'))\
    .withColumn('rank', F.row_number().over(Window.partitionBy('user_id').orderBy('record_date')))\
    .withColumn('consecutive_days', F.date_sub(F.col('record_date'), F.col('rank') - 1))\
    .groupBy('user_id', 'consecutive_days')\
    .agg(F.count('*').alias('counter'))\
    .filter(F.col('counter') >= 3)\
    .select('user_id')\
    .drop_duplicates()

result1.show()

+-------+
|user_id|
+-------+
|     U4|
+-------+



In [1]:
spark.stop()

NameError: name 'spark' is not defined